In [2]:
import quartz

In [3]:
quartz_context = quartz.QuartzContext(gate_set=['h', 'cx', 'x', 'rz', 'add'], filename='ecc_set/nam.json.ecc')
init_graph = quartz.PyGraph.from_qasm(context=quartz_context, filename='nam_circs/barenco_tof_3.qasm')

In [4]:
# available_xfer_matrix = init_graph.get_available_xfers_matrix(context=quartz_context)
all_nodes = init_graph.all_nodes()
cnt = 0
for node in all_nodes:
    l = init_graph.available_xfers(context=quartz_context, node=node)
    print(l)
    cnt += len(l)
cnt

[80, 82, 8668]
[98, 100, 8668]
[94, 210, 8668]
[98, 100, 8668]
[94, 210, 8668]
[98, 100, 2617, 2637, 2665, 8668]
[94, 210, 2019, 2023, 2769, 7026, 7027, 8668]
[98, 100, 8320, 8321, 8322, 8323, 8324, 8325, 8356, 8357, 8358, 8359, 8360, 8361, 8368, 8369, 8370, 8371, 8372, 8373, 8668]
[98, 100, 188, 189, 499, 501, 507, 511, 8668]
[94, 210, 8668]
[94, 206, 208, 210, 212, 8668]
[96, 98, 100, 102, 8668]
[97, 103, 8668]
[169, 174, 8668]
[80, 82, 8668]
[98, 100, 8668]
[94, 210, 8668]
[98, 100, 8668]
[94, 210, 8668]
[98, 100, 2617, 2637, 2665, 8668]
[94, 210, 2019, 2023, 2769, 7026, 7027, 8668]
[98, 100, 8320, 8321, 8322, 8323, 8324, 8325, 8356, 8357, 8358, 8359, 8360, 8361, 8368, 8369, 8370, 8371, 8372, 8373, 8668]
[98, 100, 188, 189, 499, 501, 507, 511, 8668]
[8668]
[94, 206, 208, 210, 212, 8668]
[78, 84, 8668]
[96, 98, 100, 102, 8668]
[98, 100, 8668]
[97, 103, 8668]
[97, 103, 8668]
[94, 210, 8668]
[98, 100, 8668]
[94, 210, 8668]
[98, 100, 2617, 2637, 2665, 8668]
[94, 210, 2019, 2023, 2769, 7

306

In [5]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

def check(graph):
    graph.to_qasm(filename='best.qasm')
    qc_origin = QuantumCircuit.from_qasm_file('barenco_tof_3_opt_path/subst_history_39.qasm')
    qc_optimized = QuantumCircuit.from_qasm_file('best.qasm')
    return Statevector.from_instruction(qc_origin).equiv(Statevector.from_instruction(qc_optimized))

In [6]:
# Optimizing with BFS
import heapq
from concurrent.futures import ProcessPoolExecutor
import copy

candidate_hq = []
heapq.heappush(candidate_hq, init_graph)
hash_set = set()
hash_set.add(init_graph.hash())
best_graph = init_graph
best_gate_cnt = init_graph.gate_count
max_gate_cnt = 64

budget = 5_000_000

while candidate_hq != [] and budget >= 0:
    first_candidate = heapq.heappop(candidate_hq)
    all_nodes = first_candidate.all_nodes()
    
    def ax(i):
        node = all_nodes[i]
        return first_candidate.available_xfers(context=quartz_context, node=node)
    
    with ProcessPoolExecutor(max_workers=64) as executor:
        results = executor.map(ax, list(range(len(all_nodes))), chunksize=2)
        appliable_xfers_nodes = []
        for r in results:
            appliable_xfers_nodes.append(r)
        
    for i in range(len(all_nodes)):
        node = all_nodes[i]
        appliable_xfers = appliable_xfers_nodes[i]
        for xfer in appliable_xfers:
            new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
            if not check(new_graph):
                print('Error')
            new_hash = new_graph.hash()
            if new_hash not in hash_set:
                hash_set.add(new_hash)
                new_cnt = new_graph.gate_count
                if new_cnt <= max_gate_cnt:
                    heapq.heappush(candidate_hq, new_graph)
                if new_cnt < best_gate_cnt:
                    best_graph = new_graph
                    best_gate_cnt = new_cnt
                budget -= 1
                if budget % 10_000 == 0:
                    print(f'{budget}: minimum gate count is {best_gate_cnt}, correctness: {check(best_graph)}')
